In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models

from skimage import io, transform

import matplotlib.pyplot as plt
import numpy as np

In [2]:
''' Ideas-
    Try to use densenet instead of resnet 18 as well and see the accuracy improvements
'''

''' Parameters to be used-
    op_vec_size : for size of output vector, to be given input to RNN
'''
op_vec_size = 10

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.get_device_name(0)

cuda:0


'NVIDIA GeForce GTX 1660 Ti with Max-Q Design'

In [4]:
dir(models)

['AlexNet',
 'DenseNet',
 'GoogLeNet',
 'GoogLeNetOutputs',
 'Inception3',
 'InceptionOutputs',
 'MNASNet',
 'MobileNetV2',
 'MobileNetV3',
 'ResNet',
 'ShuffleNetV2',
 'SqueezeNet',
 'VGG',
 '_GoogLeNetOutputs',
 '_InceptionOutputs',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_utils',
 'alexnet',
 'densenet',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'detection',
 'googlenet',
 'inception',
 'inception_v3',
 'mnasnet',
 'mnasnet0_5',
 'mnasnet0_75',
 'mnasnet1_0',
 'mnasnet1_3',
 'mobilenet',
 'mobilenet_v2',
 'mobilenet_v3_large',
 'mobilenet_v3_small',
 'mobilenetv2',
 'mobilenetv3',
 'quantization',
 'resnet',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnext101_32x8d',
 'resnext50_32x4d',
 'segmentation',
 'shufflenet_v2_x0_5',
 'shufflenet_v2_x1_0',
 'shufflenet_v2_x1_5',
 'shufflenet_v2_x2_0',
 'shufflenetv2',
 'squeezenet',
 'squeezenet1_0',
 '

In [5]:
res_net_18 = models.resnet18(pretrained=True)

In [6]:
res_net_18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
res_net_18.conv1

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [40]:
res_net_18.fc = nn.Linear(512, 10)

In [ ]:
res_net_18(img)

In [10]:
# res_net_18.fc = nn.Linear(512, op_vec_size)

In [ ]:
def equilength(dict):
    text_len=0
    #input is caption dictionary
    for id, text in dict.items():
           x=text.split()
           if (text_len < len(x)):
                    text_len = len(x)
        
        return text_len

In [22]:
class MyModelA(nn.Module):
    def __init__(self):
        super(MyModelA, self).__init__()
        self.fc1 = nn.Linear(20, 10)
        self.fc2 = nn.Linear(10, 20)
        self.fc3 = nn.Linear(20, 5)
        
        #Layers for RNN
        
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        
        #pass to RNN layers--
        
        
        return x
    

class MyModelB(nn.Module):
    def __init__(self):
        super(MyModelB, self).__init__()
        self.fc1 = nn.Linear(10, 2)
        
    def forward(self, x):
        x = self.fc1(x)
        return x


class MyEnsemble(nn.Module):
    def __init__(self, modelA, modelB):
        super(MyEnsemble, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        #self.classifier = nn.Linear(4, 2)
        
    def forward(self, x):
        x = self.modelA(x)
        op = self.modelB(x)
        #x = torch.cat((x1, x2), dim=1)
        #x = self.classifier(F.relu(x))
        return op

# Create models and load state_dicts    
modelA = MyModelA()
modelB = MyModelB()
# Load state dicts
#modelA.load_state_dict(torch.load(PATH))
#modelB.load_state_dict(torch.load(PATH))

# model_comb = MyEnsemble(modelA, modelB)
# x1 = torch.randn(1,20)
# output = model_comb(x1)

In [18]:
output

tensor([[-0.2535, -0.3676]], grad_fn=<AddmmBackward>)

In [25]:
x = torch.randn(20)
x

tensor([ 0.3824,  0.7855, -0.6723, -1.2329,  0.9496, -1.3232, -0.4424,  0.4849,
         1.7747, -0.9446, -0.0720,  0.5786,  1.2484,  1.1540, -0.7420,  1.1647,
         0.7373, -0.2458, -0.2106, -1.5648])

In [26]:
'''Laerning section for above
'''
model_a = MyModelA()

model_a(x)

tensor([ 0.0282, -0.3249,  0.2554, -0.0423, -0.4804], grad_fn=<AddBackward0>)

In [28]:
model_a.parameters

<bound method Module.parameters of MyModelA(
  (fc1): Linear(in_features=20, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=5, bias=True)
)>